# Phonon computations

This is a quick sketch how to run a simple phonon calculation using DFTK.

> **Preliminary implementation**
>
> Practical phonon computations have only seen rudimentary testing as of now.
> As of now we do not yet recommend relying on this feature for production
> calculations. We appreciate any issues, bug reports or PRs.

First we run an SCF calculation.

In [1]:
using AtomsBuilder
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(bulk(:Si); pseudopotentials, functionals=LDA())
basis  = PlaneWaveBasis(model; Ecut=10, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.916455725259                   -0.69    4.9    101ms
  2   -7.921212304052       -2.32       -1.52    1.0   51.3ms
  3   -7.921404997498       -3.72       -2.49    1.2   73.3ms
  4   -7.921440917055       -4.44       -2.82    2.5   70.7ms
  5   -7.921441567594       -6.19       -2.97    1.1   52.6ms
  6   -7.921442005955       -6.36       -4.46    1.0   51.5ms
  7   -7.921442021751       -7.80       -4.57    2.8   76.5ms
  8   -7.921442022127       -9.42       -5.26    1.0   64.2ms
  9   -7.921442022128      -12.07       -5.04    2.0   62.2ms
 10   -7.921442022143      -10.82       -5.80    1.0   52.4ms
 11   -7.921442022144      -12.12       -7.13    1.0   52.1ms
 12   -7.921442022144      -13.24       -6.49    3.2   85.4ms
 13   -7.921442022144      -13.82       -7.38    1.0   52.3ms
 14   -7.921442022144      -14.75       -7.72    1.0   98.0ms
 15   -7.

Next we compute the phonon modes at the q-point `[1/4, 1/4, 1/4]`.

In [2]:
scfres = DFTK.unfold_bz(scfres)
phret_q0 = @time DFTK.phonon_modes(scfres; q=[0.25, 0.25, 0.25]);
nothing  # hide

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   2.69s  Non-interacting
   1        0       1        0.31     52.4   18.0s  
   2        0       2       -0.52     47.0   1.85s  
   3        0       3       -2.40     41.7   1.33s  
   4        0       4       -4.46     29.7   1.03s  
   5        0       5       -6.46     17.4   745ms  
   6        0       6       -8.91      4.0   408ms  
   7        1       1       -7.84     55.5   2.06s  Restart
   8        1       2       -8.95      5.6   459ms  
                                      72.8   1.97s  Final orbitals
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      72.9   1.76s  Non-interacting
   1        0       1        0.31     52.4   2.01s  
   2        0       2       -0.52     47.0   1.44s  
   3

These are the final phonon frequencies:

In [3]:
phret_q0.frequencies

6-element Vector{Float64}:
 -0.003498155357541749
 -0.002977451158409315
 -0.0029774511582851067
  0.004302230630232087
  0.004302230630319907
  0.004353201760142032